In [1]:
import FiniteSpace_Examples as FE
import FiniteSpaces_Class as FS
import Brute_gcat as B

from itertools import combinations
from imp import reload
import numpy as np
from IPython.display import display, clear_output
import networkx as nx
import operator
import random
import time

In [2]:
# From https://codereview.stackexchange.com/questions/1526/finding-all-k-subset-partitions

In [3]:
def algorithm_u(ns, m):
    def visit(n, a):
        ps = [[] for i in range(m)]
        for j in range(n):
            ps[a[j + 1]].append(ns[j])
        return ps

    def f(mu, nu, sigma, n, a):
        if mu == 2:
            yield visit(n, a)
        else:
            for v in f(mu - 1, nu - 1, (mu + sigma) % 2, n, a):
                yield v
        if nu == mu + 1:
            a[mu] = mu - 1
            yield visit(n, a)
            while a[nu] > 0:
                a[nu] = a[nu] - 1
                yield visit(n, a)
        elif nu > mu + 1:
            if (mu + sigma) % 2 == 1:
                a[nu - 1] = mu - 1
            else:
                a[mu] = mu - 1
            if (a[nu] + sigma) % 2 == 1:
                for v in b(mu, nu - 1, 0, n, a):
                    yield v
            else:
                for v in f(mu, nu - 1, 0, n, a):
                    yield v
            while a[nu] > 0:
                a[nu] = a[nu] - 1
                if (a[nu] + sigma) % 2 == 1:
                    for v in b(mu, nu - 1, 0, n, a):
                        yield v
                else:
                    for v in f(mu, nu - 1, 0, n, a):
                        yield v

    def b(mu, nu, sigma, n, a):
        if nu == mu + 1:
            while a[nu] < mu - 1:
                yield visit(n, a)
                a[nu] = a[nu] + 1
            yield visit(n, a)
            a[mu] = 0
        elif nu > mu + 1:
            if (a[nu] + sigma) % 2 == 1:
                for v in f(mu, nu - 1, 0, n, a):
                    yield v
            else:
                for v in b(mu, nu - 1, 0, n, a):
                    yield v
            while a[nu] < mu - 1:
                a[nu] = a[nu] + 1
                if (a[nu] + sigma) % 2 == 1:
                    for v in f(mu, nu - 1, 0, n, a):
                        yield v
                else:
                    for v in b(mu, nu - 1, 0, n, a):
                        yield v
            if (mu + sigma) % 2 == 1:
                a[nu - 1] = 0
            else:
                a[mu] = 0
        if mu == 2:
            yield visit(n, a)
        else:
            for v in b(mu - 1, nu - 1, (mu + sigma) % 2, n, a):
                yield v

    n = len(ns)
    a = [0] * (n + 1)
    for j in range(1, m + 1):
        a[n - m + j] = j - 1
    return f(m, n, 0, n, a)

In [4]:
def pretty_print(parts):
    print ('; '.join('|'.join(''.join(str(e) for e in loe) for loe in part) for part in parts))

def count(parts):
    i = 0
    for e in parts:
        i+=1
    return i

In [5]:
pretty_print(algorithm_u([1, 2, 3, 4], 3))
count(algorithm_u([1, 2, 3, 4], 3))

12|3|4; 1|23|4; 13|2|4; 1|2|34; 1|24|3; 14|2|3


6

In [8]:
out  = algorithm_u([1, 2, 3, 4], 3)
count(out)

6

In [9]:
def partition(items, k):
    '''
    Code from:
    https://codereview.stackexchange.com/questions/1526/finding-all-k-subset-partitions

    Uses Algorithm U, described by Knuth in the Art of Computer Programming, Volume 4,
    Fascicle 3B.

    Parameters
    ----------
    items : a list of elements
    k : an integer, the number of sets in the partition

    Yields
    ------
    An iterator of all possible partitionings of size $k$ of the elements.

    '''
    m = k

    def visit(n, a):
        ps = [[] for i in range(m)]
        for j in range(n):
            ps[a[j + 1]].append(items[j])
        return ps

    def f(mu, nu, sigma, n, a):
        if mu == 2:
            yield visit(n, a)
        else:
            for v in f(mu - 1, nu - 1, (mu + sigma) % 2, n, a):
                yield v
        if nu == mu + 1:
            a[mu] = mu - 1
            yield visit(n, a)
            while a[nu] > 0:
                a[nu] = a[nu] - 1
                yield visit(n, a)
        elif nu > mu + 1:
            if (mu + sigma) % 2 == 1:
                a[nu - 1] = mu - 1
            else:
                a[mu] = mu - 1
            if (a[nu] + sigma) % 2 == 1:
                for v in b(mu, nu - 1, 0, n, a):
                    yield v
            else:
                for v in f(mu, nu - 1, 0, n, a):
                    yield v
            while a[nu] > 0:
                a[nu] = a[nu] - 1
                if (a[nu] + sigma) % 2 == 1:
                    for v in b(mu, nu - 1, 0, n, a):
                        yield v
                else:
                    for v in f(mu, nu - 1, 0, n, a):
                        yield v

    def b(mu, nu, sigma, n, a):
        if nu == mu + 1:
            while a[nu] < mu - 1:
                yield visit(n, a)
                a[nu] = a[nu] + 1
            yield visit(n, a)
            a[mu] = 0
        elif nu > mu + 1:
            if (a[nu] + sigma) % 2 == 1:
                for v in f(mu, nu - 1, 0, n, a):
                    yield v
            else:
                for v in b(mu, nu - 1, 0, n, a):
                    yield v
            while a[nu] < mu - 1:
                a[nu] = a[nu] + 1
                if (a[nu] + sigma) % 2 == 1:
                    for v in f(mu, nu - 1, 0, n, a):
                        yield v
                else:
                    for v in b(mu, nu - 1, 0, n, a):
                        yield v
            if (mu + sigma) % 2 == 1:
                a[nu - 1] = 0
            else:
                a[mu] = 0
        if mu == 2:
            yield visit(n, a)
        else:
            for v in b(mu - 1, nu - 1, (mu + sigma) % 2, n, a):
                yield v

    n = len(items)
    a = [0] * (n + 1)
    for j in range(1, m + 1):
        a[n - m + j] = j - 1
    return f(m, n, 0, n, a)


In [10]:
for e in partition(['a','b','c'],2):
    print(e)

[['a', 'b'], ['c']]
[['a'], ['b', 'c']]
[['a', 'c'], ['b']]


In [14]:
for e in partition(list(range(25)),1):
    print(e)

RecursionError: maximum recursion depth exceeded in comparison

In [13]:
count(partition(list(range(25)),24))

300

In [46]:
maxs = list(range(10))
verbose = True
def test(maxs):
    gc = len(maxs)
    
    found = True
    k = gc 
    
    while found and k >= 2: 
        print('k=',k)
        # Start checking for a partition of size $k$ that is a valid cover
        if k>=2:
            for part in partition(list(maxs), k):
    #             print(part)
                if [1] in part:
                    # If you're in here, you have a valid cover.
                    print(part)
                    gc = len(part)
                    k -=1 
                    if verbose:
                        print('\tFound cover of size', gc)
                    break  
                
        # If you didn't break, you didn't find a cover
        if gc != k+1:
            found = False
            print('\tNo cover found of size', len(part),'... exiting.')
            return gc
        
        
    if gc == 2:
        # If you got down to 2, test 1
        print('k = 1')
        part = [maxs]
        if [1] in part:
            print(part)
            gc = len(part)
            print('\tFound cover of size', gc)
        else:
            print('\tNo cover found of size', len(part),'... exiting.')
            return gc
        
    return gc

test(maxs)

k= 10
[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]
	Found cover of size 10
k= 9
[[0, 2], [1], [3], [4], [5], [6], [7], [8], [9]]
	Found cover of size 9
k= 8
[[0, 2, 3], [1], [4], [5], [6], [7], [8], [9]]
	Found cover of size 8
k= 7
[[0, 2, 3, 4], [1], [5], [6], [7], [8], [9]]
	Found cover of size 7
k= 6
[[0, 2, 3, 4, 5], [1], [6], [7], [8], [9]]
	Found cover of size 6
k= 5
[[0, 2, 3, 4, 5, 6], [1], [7], [8], [9]]
	Found cover of size 5
k= 4
[[0, 2, 3, 4, 5, 6, 7], [1], [8], [9]]
	Found cover of size 4
k= 3
[[0, 2, 3, 4, 5, 6, 7, 8], [1], [9]]
	Found cover of size 3
k= 2
[[0, 2, 3, 4, 5, 6, 7, 8, 9], [1]]
	Found cover of size 2
k = 1
	No cover found of size 1 ... exiting.


2